project name: flexpart_management
created by diego aliaga daliaga_at_chacaltaya.edu.bo

imports

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from useful_scit.imps import *
# noinspection PyUnresolvedReferences
import matplotlib.colors
import flexpart_management.modules.FlexLogPol as FlexLogPol
import flexpart_management.modules.constants as co
# noinspection PyUnresolvedReferences
import flexpart_management.modules.flx_array as fa
import flexpart_management.modules.clustering_funs as cfuns
from sklearn.metrics import silhouette_samples

In [7]:
ds = xr.open_mfdataset( [ co.latest_ds_mac ] , combine='by_coords' )

In [8]:
ds

<xarray.Dataset>
Dimensions:                             (R_CENTER: 31, TH_CENTER: 36, ZMID: 30, releases: 4248)
Coordinates:
  * TH_CENTER                           (TH_CENTER) float64 0.08727 ... 6.196
  * releases                            (releases) datetime64[ns] 2017-12-06 ... 2018-05-31T23:00:00
  * R_CENTER                            (R_CENTER) float64 0.08046 ... 17.81
  * ZMID                                (ZMID) float64 250.0 750.0 ... 1.475e+04
    ZBOT                                (ZMID) float64 dask.array<chunksize=(30,), meta=np.ndarray>
    ZLEN_M                              (ZMID) float64 dask.array<chunksize=(30,), meta=np.ndarray>
    ZTOP                                (ZMID) float64 dask.array<chunksize=(30,), meta=np.ndarray>
    LON                                 (R_CENTER, TH_CENTER) float64 dask.array<chunksize=(31, 36), meta=np.ndarray>
    LAT_00                              (R_CENTER, TH_CENTER) float64 dask.array<chunksize=(31, 36), meta=np.ndarray>
 

In [ ]:
conc_var = 'conc_smooth_norm'
lab = 'lab'
above_thre = 'above_thre'
# labs = ds[lab]
# dsc = ds[conc_var].to_dataset(co.RL)

In [ ]:
# dsc

In [ ]:

sel = {
    co.RL:slice(None,None,1),
    co.R_CENTER:slice(None,None,1),
    co.TH_CENTER:slice(None,None,1),
    co.ZM:slice(None,None,1),
}
ds_reset = ds.reset_coords()[[conc_var, lab, above_thre]]
ds_reset = ds_reset[sel].assign_coords({lab:ds_reset[lab]})
ds_new = ds_reset.where(ds_reset[above_thre]).drop(above_thre)
da = ds_new[conc_var].drop(lab)
df = da.load().to_dataframe()[conc_var].dropna()
dfu = df.unstack(co.RL)

In [ ]:
dl = ds_new[lab].reset_coords(drop=True)
dlf =dl.to_dataframe()
dlf

In [ ]:

index = dfu.index
vals = dfu.loc[index]
labs = dlf.loc[index]

In [ ]:

scaler = QuantileTransformer()
vals_quan = scaler.fit_transform( vals.T ).T * 1000
# vals_quan = vals*1000

In [ ]:
tot = vals.sum(axis=1)
plt.hist(tot)
plt.hist(vals_quan.sum(axis=1))
plt.show()

In [ ]:
out_dic = {}
for n in range(2,25):
    print(n)
    from sklearn.cluster import KMeans
    new_lab = KMeans(n_clusters=n).fit_predict(vals_quan,sample_weight=100*tot)
    res = silhouette_samples(vals_quan, new_lab)
    from sklearn.metrics import silhouette_score

    # print(silhouette_score(vals_quan, new_lab))
    wss = (res*tot).sum()/tot.sum()
    ss = res.mean()
    print(wss)
    print(ss)
    dic = {'ss':ss,'wss':wss,'res':res,'new_lab':new_lab}
    out_dic[n]=dic

In [ ]:
dlf['res']=np.nan
dlf['tot']=np.nan
dlf.loc[index,'res'] = res
dlf.loc[index,'tot'] = vals.sum(axis=1)

In [ ]:
ndf = dlf.loc[index].sort_values(['lab','res']).reset_index()
(ndf['res']*ndf['tot']).plot()
plt.show()

In [1]:
import numpy as np
a=0.08046 
b=0.09633 
c=0.1153

In [13]:
al=np.log(a)
bl=np.log(b)
cl=np.log(c)

In [14]:
bl-al

0.18001963024789003

In [15]:
cl-bl

0.1797576305054811

In [16]:
np.exp(bl+.18)

0.11532794858952398

In [12]:
np.exp(np.log(3))

3.0000000000000004

In [17]:
 4248 * 31 *36 * 30

142223040

In [18]:
 31 *36 * 30

33480